In [16]:
import torch
import pytorch_lightning as pl
import importlib
import numpy as np
# from triplet_vade import TripletVaDE
import triplet_vade
importlib.reload(triplet_vade)
from autoencoder import SimpleAutoencoder, VaDE, ClusteringEvaluationCallback, cluster_acc


In [18]:
model = SimpleAutoencoder(n_neurons=[784, 512, 512, 2048, 10], lr=1e-3, batch_size=2**10)

logger =  pl.loggers.wandb.WandbLogger(project='AE clustering')
callbacks = [ClusteringEvaluationCallback(on_start=False), pl.callbacks.LearningRateMonitor()]
trainer = pl.Trainer(gpus=1, callbacks=[ClusteringEvaluationCallback(on_start=False), pl.callbacks.LearningRateMonitor()],
                    logger=logger, log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=100)
model.load_state_dict(torch.load('trained_models/AE1.h5'))
# trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [25]:
model.encoder[:-1].state_dict().keys() , triplets_model.model.encoder.state_dict().keys()

(odict_keys(['0.0.weight', '0.0.bias', '1.0.weight', '1.0.bias', '2.0.weight', '2.0.bias']),
 odict_keys(['0.0.weight', '0.0.bias', '0.2.weight', '0.2.bias', '0.2.running_mean', '0.2.running_var', '0.2.num_batches_tracked', '1.0.weight', '1.0.bias', '1.2.weight', '1.2.bias', '1.2.running_mean', '1.2.running_var', '1.2.num_batches_tracked', '2.0.weight', '2.0.bias', '2.2.weight', '2.2.bias', '2.2.running_mean', '2.2.running_var', '2.2.num_batches_tracked']))

In [23]:
triplets_model = triplet_vade.TripletVaDE(n_neurons=[784, 512, 512, 2048, 10], lr=3e-3, batch_size=2**10, pretrain_epochs=15)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 1.7 M 
1 | decoder | Sequential | 1.7 M 
---------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
logger =  pl.loggers.wandb.WandbLogger(project='VaDE Triplets')
callbacks = [ClusteringEvaluationCallback(on_start=True), pl.callbacks.LearningRateMonitor()]
trainer = pl.Trainer(gpus=1, callbacks=[ClusteringEvaluationCallback(on_start=False), pl.callbacks.LearningRateMonitor()],
                    logger=logger, log_every_n_steps=10, progress_bar_refresh_rate=10, max_epochs=100)


In [4]:
# torch.save(model.state_dict(), 'trained_models/AE1.h5')

In [19]:
model.load_state_dict(torch.load('trained_models/AE1.h5'))

<All keys matched successfully>

In [8]:
from sklearn import metrics

model.cuda()
true_y, predicted_y, X_encoded = model.cluster_data(method='kmeans')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-full')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

true_y, predicted_y, X_encoded = model.cluster_data(method='gmm-diag')
print(metrics.normalized_mutual_info_score(true_y, predicted_y))

0.7538434361452869
0.8249112629202103
0.7238449211641871


In [10]:
from sklearn.mixture import GaussianMixture
scores = []
for i in range(5):
    gmm = GaussianMixture(n_components=10, covariance_type='full', n_init=5)
    predicted_y = gmm.fit_predict(X_encoded)
    scores.append(metrics.normalized_mutual_info_score(true_y, predicted_y))
    
print(scores)
print(np.mean(scores), np.max(scores), np.std(scores))

[0.8206645765011308, 0.8503932657020613, 0.7995011494626447, 0.8239846028460401, 0.8483506486929793]
0.8285788486409713 0.8503932657020613 0.018952826999921197
